[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JDekun/MoCo/blob/master/MoCo.ipynb?authuser=2) [![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/code/mydekun/MoCo) [![Open In Github](https://badgen.net/badge/icon/Open%20MoCo?icon=github&label)](https://github.com/JDekun/MoCo/blob/master/MoCo.ipynb)

In [1]:
#@title [Colab] 构建Kaggle环境 && 下载数据集

'''
自动化代码，当值 colab 断路
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
'''
# clearInterval()

######### 数据集名字 #########
datasets = ["vijayabhaskar96/pascal-voc-2007-and-2012"]  #@param {type:"raw"}
import os

# 构建 working input 文件夹和文件
if not os.path.exists('/content/working'):
  !mkdir /content/working
if not os.path.exists('/content/input'):
  !mkdir /content/input

# 导入Kaggle API
os.chdir('/content')
!git clone https://github.com/JDekun/Kaggle.git > /dev/null 2>&1
os.environ['KAGGLE_CONFIG_DIR'] = '/content/Kaggle' #注意kaggle文件夹包含json文件 

# 下载数据集
os.chdir('/content/input')
len_mydekun = len(datasets)
if len_mydekun != 0:
  for i in range(len_mydekun):
    # 下载 kaggle 数据集
    temp = datasets[i]
    name, dataset = temp.split('/')
    dirlist = os.listdir('/content/input')

    if (dataset in dirlist):
      print("您已经下载过'%s'数据集" % dataset)
    else:
      zip = dataset + '.zip'
      !kaggle datasets download -d $temp -p $dataset
      # 解压数据集并删除压缩包
      !unzip $dataset/$zip -d $dataset > /dev/null 2>&1
      !rm -f $dataset/$zip

 99% 2.63G/2.64G [00:17<00:00, 178MB/s]
100% 2.64G/2.64G [00:17<00:00, 163MB/s]


In [2]:
#@title [Kaggle] 克隆github项目
import os
######### GitHub库 #########
github = "https://github.com/JDekun/MoCo.git"  #@param {type:"string"}
## 工作路径
working = github.rsplit('/',1)[-1].rsplit('.',1)[0]

try:
  os.chdir("/kaggle/working/")
except:
  os.chdir("/content/working/")

!git clone $github   # 克隆 github 项目

os.chdir(working)

Cloning into 'MoCo'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 58 (delta 9), reused 18 (delta 7), pack-reused 38
Unpacking objects: 100% (58/58), done.


In [3]:
#@title [TensorBoard] 在 Colab 中启动TensorBoard
path = ""  #@param {type: "string"}
len_tbp = len(path)
if len_tbp != 0:
  %reload_ext tensorboard
  %tensorboard --logdir $path

In [4]:
os.chdir('./detection')

1.Install detectron2.

In [5]:
!python -m pip install pyyaml==5.1 > /dev/null
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' > /dev/null

  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-o83fycya
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


2.Convert a pre-trained MoCo model to detectron2's format:

In [6]:
!wget https://dl.fbaipublicfiles.com/moco/moco_checkpoints/moco_v1_200ep/moco_v1_200ep_pretrain.pth.tar > /dev/null
!python3 convert-pretrain-to-detectron2.py moco_v1_200ep_pretrain.pth.tar output.pkl

--2022-08-10 11:17:32--  https://dl.fbaipublicfiles.com/moco/moco_checkpoints/moco_v1_200ep/moco_v1_200ep_pretrain.pth.tar
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95347404 (91M) [application/octet-stream]
Saving to: ‘moco_v1_200ep_pretrain.pth.tar’

moco_v1_200ep_pretr 100%[===================>]  90.93M  23.1MB/s    in 7.4s    

2022-08-10 11:17:40 (12.3 MB/s) - ‘moco_v1_200ep_pretrain.pth.tar’ saved [95347404/95347404]

module.encoder_q.conv1.weight -> stem.conv1.weight
module.encoder_q.bn1.weight -> stem.conv1.norm.weight
module.encoder_q.bn1.bias -> stem.conv1.norm.bias
module.encoder_q.bn1.running_mean -> stem.conv1.norm.running_mean
module.encoder_q.bn1.running_var -> stem.conv1.norm.running_var
module.encoder_q.bn1.num_batches_tracked -> stem.conv1.norm.num_batches_t

3.Put dataset under "./datasets" directory

In [40]:
!export DETECTRON2_DATASETS='./datasets/da'

4.Run training:

In [ ]:
!python train_net.py --config-file configs/pascal_voc_R_50_C4_24k_moco.yaml MODEL.WEIGHTS ./output.pkl